# Calculating Barriers in Metal - Semiconductor junctions 
- all calculations indepandant of the material
- for both p and n-type


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import os
import pandas as pd

### Physical Constants

In [4]:
Q = 1.602e-19  # Elementary charge (C)
EPSILON_0 = 8.854e-14  # Permittivity of free space (As/Vcm) -> in cm here to math the doping concentration units
K_B_eV = 8.617e-5  # Boltzmann constant (eV/K)
K_B_J = 1.380649e-23  # Boltzmann constant (J/K)
T = 300  # Temperature (K)
h = 6.626e-34# planck constant (Js)

### Material Parameters Class
allows for easier swaps of materials

In [14]:
class MaterialParameters:
    """Stores and manages the material parameters for the semiconductor and metal."""
    def __init__(self, name, type, phi_s, chi, Nd_or_Na, epsilon_r, Nc=None, Nv=None):
        """
        :param name: Material name (e.g., 'Silicon', 'Aluminium').
        :param type: 'n' or 'p' for semiconductor, 'metal' for metal.
        :param phi_s: Work function (eV) for metal OR band gap (Eg) for semiconductor.
        :param chi: Electron Affinity (eV) for semiconductor.
        :param Nd_or_Na: Donor (Nd) or Acceptor (Na) concentration (atoms/cm^3) for semiconductor.
        :param epsilon_r: Relative permittivity (dielectric constant).
        :param Nc: Effective Density of States in Conduction Band (cm^-3).
        :param Nv: Effective Density of States in Valence Band (cm^-3).
        """
        self.name = name
        self.type = type
        self.phi_s = phi_s # Work function or Eg
        self.chi = chi
        self.Nd_or_Na = Nd_or_Na
        self.epsilon_r = epsilon_r
        # Calculate absolute permittivity (F/cm)
        if self.epsilon_r is not None: self.epsilon = self.epsilon_r * EPSILON_0
        
        self.Nc = Nc
        self.Nv = Nv

### Junction Calculation Class
- ideal barrier
- fermi level pinning
- depletion width

In [21]:
class JunctionCalculator:
    """Performs calculations for the metal-semiconductor junction."""

    def __init__(self, metal, semiconductor):
        self.metal = metal
        self.semiconductor = semiconductor
        self.type = semiconductor.type.lower() # 'n' or 'p'


    def _calculate_fermi_level_position(self):
        """
        Calculates the Fermi level position relative to the band edge (in eV).
        This result is E_c - E_F for n-type or E_F - E_v for p-type.
        """
        kBT_over_q = K_B_eV * T # k_B T/q (in Volts)

        if self.type == 'n':
            # E_C-E_F = k_B T/q * ln (N_D/N_C) (in V)
                        
            # NOTE: We must ensure N_D < N_C for the Boltzmann approximation to be valid
            N_D = self.semiconductor.Nd_or_Na
            N_C = self.semiconductor.Nc
            
            if N_D > N_C:
                # Fermi level is in the conduction band (degenerate case)
                print("n--- WARNING: Degenerate doping (ND > NC). Result may be inaccurate. ---")
            
            V_n = kBT_over_q * np.log(self.semiconductor.Nc / N_D)
            return V_n
            
        elif self.type == 'p':
            # E_F - E_V = k_B T/q * ln(N_A/N_V) (in V)
            N_A = self.semiconductor.Nd_or_Na
            N_V = self.semiconductor.Nv

            if N_A > N_V:
                print("n--- WARNING: Degenerate doping (NA > NV). Result may be inaccurate. ---")
                
            V_p = kBT_over_q * np.log(self.semiconductor.Nv / N_A)
            return V_p
        
        else:
            raise ValueError("Semiconductor type must be 'n' or 'p'.")


    def _calculate_v_bi(self):
            """
            Calculates the built-in potential (V_bi) based on the Schottky-Mott model:
            V_bi = 1/q * (Phi_m - Phi_s), where Phi_s is the semiconductor work function.
            
            V_bi (n-type) = Phi_m - chi - V_n       (in Volts)
            V_bi (p-type) = Phi_m - chi - E_g + V_p (in Volts)
            
            Where Phi_m, chi, E_g are in eV and V_n, V_p are in Volts.
            Since V_n and V_p are already calculated in Volts, the formula simplifies.
            """
            phi_m = self.metal.phi_s        # Phi_m (eV)
            chi = self.semiconductor.chi    # chi (eV)
            Eg = self.semiconductor.phi_s   # E_g (eV)

            # Fermi level position relative to band edge, V_n or V_p (in Volts)
            V_fermi_band_edge = self._calculate_fermi_level_position() 

            if self.type == 'n':
                # The semiconductor work function is Phi_s = chi + (E_c - E_F)/q = chi + V_n
                # V_bi = (Phi_m - Phi_s)/q
                # V_bi = Phi_m - chi - V_fermi_band_edge
                V_bi = phi_m - chi - V_fermi_band_edge
                
            elif self.type == 'p':
                # The semiconductor work function is Phi_s = chi + E_g - (E_F - E_v)/q = chi + E_g - V_p
                # V_bi = Phi_m - (chi + E_g - V_fermi_band_edge)
                V_bi = phi_m - (chi + Eg - V_fermi_band_edge)
            
            else:
                raise ValueError("Semiconductor type must be 'n' or 'p'.")
                
            return V_bi



    def ideal_barrier_height(self):
            """
            Calculates the ideal Schottky Barrier Height Phi_B,ideal
            """
            phi_m = self.metal.phi_s # Metal work function (eV)
            chi = self.semiconductor.chi # Electron Affinity (eV)
            
            if self.type == 'n':
                # Phi_{Bn, ideal} = Phi_m - chi
                phi_b_ideal = phi_m - chi
            elif self.type == 'p':
                Eg = self.semiconductor.phi_s # Band gap (eV)
                # Phi_{Bp, ideal} = E_g - (Phi_m - chi)
                phi_b_ideal = Eg - (phi_m - chi)
            else:
                raise ValueError("Semiconductor type must be 'n' or 'p'")

            return phi_b_ideal
        
        
    def barrier_height_pinning(self, S):
        """
        Calculates the barrier height with Fermi Level Pinning
        :param S: Pinning factor (0 < S < 1). S=1 is ideal, S=0 is full pinning.
        """
        # You will need to provide the equation here.
        # A common form includes the charge neutrality level (Phi_0 or E_{CNL})
        # Phi_B = S(Phi_m - chi) + (1-S)(Phi_0 - chi) (for n-type)
        # Where Phi_0 is the charge neutrality level relative to the vacuum level.
        
        # For simplicity, let's use a simplified linear interpolation:
        phi_b_ideal = self.ideal_barrier_height()
        # Assume Phi_{B, max_pinning} is the barrier height when S=0 (full pinning).
        # You need the value for Phi_{B, max_pinning} or Phi_0.
        
        # Placeholder for Phi_{B, max_pinning} (e.g., E_g/2 for Si)
        Eg = self.semiconductor.phi_s
        # Let's use a typical value for the charge neutrality level, Phi_0.
        # For Si, Phi_0 approx 4.7 eV (relative to vacuum, E_{CNL} is relative to E_v).
        # Let's assume a simplified Phi_{B, S=0} value for the structure:
        
        if self.type == 'n':
            # Phi_{Bn} = S Phi_{Bn, ideal} + (1-S) Phi_{Bn, pinned}
            # Where Phi_{Bn, pinned} is the pinned barrier height (e.g., E_g - E_{CNL} relative to E_c)
            phi_b_pinned = Eg * 0.5 # Simplification: Assume pinning to mid-gap
        elif self.type == 'p':
            # Phi_{Bp} = S Phi_{Bp, ideal} + (1-S) Phi_{Bp, pinned}
            phi_b_pinned = Eg * 0.5 # Simplification: Assume pinning to mid-gap

        phi_b_pinning = S * phi_b_ideal + (1 - S) * phi_b_pinned

        return phi_b_pinning


    def depletion_width(self, V_ext=0):
            """
            Calculates the depletion width (W) at a given applied voltage (V_ext).
            Requires the built-in voltage (V_bi).
            """
            V_bi = self._calculate_v_bi()
            N = self.semiconductor.Nd_or_Na
            epsilon = self.semiconductor.epsilon

            # W = sqrt{2 epsilon (V_bi - V_a)/Q N}
            try:
                W = np.sqrt((2 * epsilon * (V_bi - V_ext)) / (Q * N))
            except ValueError:
                print("Error: (V_bi - V_ext) is negative. Check inputs or V_bi calculation.")
                W = 0
                
            return W * 1e7 # Convert cm to nm

# Testing

In [28]:
# def SiC parameters
n_SiC = MaterialParameters(
    name='n_SiC',
    type='n',
    phi_s=3.26,
    chi=3.2,
    Nd_or_Na=4.33e17,
    epsilon_r=9.7,
    Nc=1.7e19,
    Nv=2.5e19)

Ni2Si = MaterialParameters(
    name='Ni2Si',
    type='metal',
    phi_s=4.8,
    chi=None,
    Nd_or_Na=None,
    epsilon_r=None)

print(n_SiC.__dict__)
print(Ni2Si.__dict__)

{'name': 'n_SiC', 'type': 'n', 'phi_s': 3.26, 'chi': 3.2, 'Nd_or_Na': 4.33e+17, 'epsilon_r': 9.7, 'epsilon': 8.588379999999999e-13, 'Nc': 1.7e+19, 'Nv': 2.5e+19}
{'name': 'Ni2Si', 'type': 'metal', 'phi_s': 4.8, 'chi': None, 'Nd_or_Na': None, 'epsilon_r': None, 'Nc': None, 'Nv': None}


In [27]:
phi_m = JunctionCalculator(Ni2Si, n_SiC).ideal_barrier_height()
print(phi_m, ' eV')

w_scr = JunctionCalculator(Ni2Si, n_SiC).depletion_width()
print(w_scr, ' nm')

1.5999999999999996  eV
61.04935885535295  nm
